## Gradient based optimization methods

In [ ]:
# попробовать изображение с несколькими обьектами
# соты
# соты с дырками

In [ ]:
#import notebook_importer
#from utility import *
import cv2
import random 
import numpy as np
from numpy import sqrt, sum, abs, max, maximum, logspace, exp, log, log10, zeros
from numpy.linalg import norm
from numpy.random import randn, rand
import urllib
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
from scipy import optimize
np.random.seed(0)

## Gradient descent

### Lipschitz Estimator
A method that estimates the Lipschitz constant of a function $g$.  This can be done using the formula
$$L \approx \frac{\|g(x) - g(y)\|}{\|x-y\|}.$$
The inputs should be a function $g:\mathbb{R}^n \to \mathbb{R}^m,$ and an initial vector $x$ in the domain of $g$.

In [ ]:
def estimate_lipschitz(g, x):
    y = rand(*x.shape)
    L = norm(g(x)-g(y))/norm(x-y)
    return L

### A routine that minimizes a function using gradient descent.
The inputs $f$ and $grad$ are function handles.  The function $f: \mathbb{R}^N\to \mathbb{R}$ is an arbitrary objective function, and  $grad: \mathbb{R}^N \to \mathbb{R}^N$ is its gradient.  The method minimizes $f$ using gradient descent, and terminate when the gradient of $f$ is small.  

**Stopping condition:**
$$\|\nabla f(x^k)\|<\|\nabla f(x^0)\|*tol$$
 where $x^0$ is an initial guess and $tol$ is a small tolerance parameter (a typical value would be $10^{-4}$).  
 
  Use a backtracking line search to guarantee convergence.   The stepsize should be monotonically decreasing.  Each iteration should begin by trying the stepsize that was used on the previous iteration, and then backtrack until the Armijo condition holds:
  $$f(x^{k+1}) \le f(x^k) + \alpha \langle x^{k+1} - x^k, \nabla f(x^k)\rangle,$$
  where $\alpha \in (0,1),$ and $\alpha=0.1$ is suggested.

  The function returns the solution vector $x_{sol}$, and also a vector $res$ containing the norm of the residual (i.e., the norm of the gradient) at each iteration.

This initial stepsize should be $10/L$, where $L$ is an estimate of the Lipschitz constant for the gradient. We over-estimate the step size intially and tone it down using the line search condition.

In [ ]:
def grad_descent(f, grad, x0, max_iters=10000, tol=1e-4):
    x_k = x0
    L = estimate_lipschitz(grad, x_k)
    step_size = 10/L
    res=[]
    res.append(norm(grad(x_k)))
    d = -grad(x0)
    while (np.linalg.norm(d)>tol):
        d = -grad(x_k)

        while(f(x_k + step_size*d)>=(f(x_k) + 0.1*(np.sum((step_size*d)*(-d))))):
            step_size = step_size/2
        x_k1 = x_k + step_size*d  
        res.append(norm((-d)))
        
        if res[-1] < tol*res[0]:
            x= x_k
            break
        x_k = x_k1     
        
    return x, res

### Gradient solver that begins each iteration using a Barzilai-Borwein stepsize (BB Method) 
  $$\tau = \frac{\langle x^{k+1} - x^k ,x^{k+1} - x^k   \rangle}{\langle x^{k+1} - x^k ,\nabla f(x^{k+1}) - \nabla f(x^k)   \rangle}.$$
 


In [ ]:
def grad_descent_bb(f, grad, x0, max_iters=10000, tol=1e-4):
    x_k = x0
    L = estimate_lipschitz(grad, x0)
    step_size = L/100
    x_k1= x_k - step_size*grad(x_k)
    res = []
    res.append(norm(grad(x_k)))
    d = -grad(x_k)
    while (np.linalg.norm(d)>tol):
        
        step_size = (np.sum((x_k1-x_k)*(x_k1 - x_k)))/(np.sum((x_k1 - x_k)*(grad(x_k1) +d)))        

        
        while(f(x_k + step_size*d)>=(f(x_k) + 0.1*step_size*(np.sum((d)*(-d))))):
            step_size = step_size/2        

        x_k = x_k1
        d = -grad(x_k)
        res.append(norm(d))
        x_k1 = x_k + step_size*d

        if(res[-1]<tol*res[0]):
            x= x_k
            break
        
    
    return x, res


### A routine that uses Nesterov's accelerated gradient method
\begin{align}
x^{k} &= y^k - \tau \nabla f(y^k)\\
\delta^{k+1} &= \frac{1+\sqrt{1+4(\delta^k)^2}}{2}\\
y^{k+1} &= x^{k}+\frac{\delta^k-1}{\delta^{k+1}}(x^k-x^{k-1})
\end{align}
The stepsize restriction for Nesterov's methods is $\tau<1/L,$ however when $L$ is not known exactly you can use the line search condition
 $$f(x^k) \le f(y^{k}) + \alpha (x^k-y^k)^T\nabla f(y^k), $$
 where $\alpha \in [1/2,1).$  

In [ ]:
def grad_descent_nesterov(f, grad, x0, max_iters=10000, tol=1e-4):
    L = estimate_lipschitz(grad, x0)
    step_size = 0.8/L
    delta_k =1 
    res = []
    y_k = x0
    x_k_prev = x0
    x_k = y_k - step_size*grad(y_k)
    res.append(norm(grad(x_k_prev)))
    while (np.linalg.norm(grad(x_k))>tol):
       
        d=-grad(y_k)
        #Armijo condition
        while(f(y_k+step_size*d)>=(f(y_k) + 0.5*(np.sum((step_size*d)*(-d))))):
            step_size = step_size/2        
        
        
        #Calculation for updates
        x_k = y_k - step_size*(-d)
        delta_k1 = (1 + np.sqrt(4*(delta_k**2) + 1))/2
        y_k1 = x_k + ((delta_k - 1)/(delta_k1))*(x_k - x_k_prev)
        
        #Updates
        delta_k = delta_k1
        x_k_prev = x_k
        y_k = y_k1
        res.append(norm(grad(x_k)))
        
        #Check for convergence
        if res[-1] < tol*res[0]:
            x= x_k
            break
      
    return x, res

## Fletcher-Reeves

In [ ]:
def FR(f,grad,x0,e=0.001):
    
    xcur = np.array(x0)
    n = len(x0)
    k = 0 # step1
    dk=grad(x0)
    prevgrad = 1
    pk = -1*dk
    res=[]
    res.append(norm(grad(x0)))
    #while (k<1000): # step3
    while (np.linalg.norm(dk)>e): # step3
        if (k%n==0): # step4
            pk = -1*dk
        else:
            bk = (np.linalg.norm(dk)**2)/(np.linalg.norm(prevgrad)**2) # step5
            prevpk = pk
            pk = -1*dk + bk*prevpk # step6
        a = (optimize.minimize_scalar(lambda x: f(xcur+pk*x), bounds=(0,10)).x)
        xcur = xcur + a*pk #step8
        k=k+1 #step8
        prevgrad=dk
        dk=grad(xcur)
        res.append(norm(dk))
    return xcur,res,k #step10

In [ ]:
def HY(f,grad,x0,e=0.001):
    xcur = np.array(x0)
    n = len(x0)
    k = 0 # step1
    dk=grad(x0)
    prevgrad = 1
    pk = -1*dk
    res=[]
    res.append(norm(grad(x0)))
    while (np.linalg.norm(dk)>e): # step3
        a = (optimize.minimize_scalar(lambda x: f(xcur+pk*x), bounds=(0,100)).x)
        xpred=xcur
        xcur = xcur + a*pk #step8
        if (k%n==0): # step4
            pk = -1*dk
        else:
            bk = (np.linalg.norm(dk)**2) / (np.dot(np.squeeze(-dk+prevgrad), np.squeeze(pk)))
            prevpk = pk
            pk = -1*dk + bk*prevpk # step6
        
        
        k=k+1 #step8
        prevgrad=dk
        dk=grad(xcur)
        res.append(norm(dk))
    return xcur,res #step10

## Image denoising


#### Total Variation (TV) objective to denoise an image

**TV objective**
$$ |\nabla x| + \frac{\mu}{2}\|x-f\|^2$$

Since L1 norm is not differentiable, we use hyperbolic regularization where
$$ h(x) = \sqrt{x^2+\epsilon ^2}$$

The objective becomes:
$$h(\nabla x) + \frac{\mu}{2}\|x-f\|^2$$

The derivative of this objective is :
$$ \nabla^{T}h'(\nabla x) + \mu(x-f)$$

where $$ h'(x) = \frac{x}{\sqrt{x^2 + \epsilon^2}}$$

In [ ]:
# Gradient of the individual components of the objective
kernel_h = [[1,-1,0]] 
kernel_v = [[1],[-1],[0]]

def gradh(x):
    """Discrete gradient/difference in horizontal direction"""
    return convolve2d(x,kernel_h, mode='same', boundary='wrap')
def gradv(x):
    """Discrete gradient/difference in vertical direction"""
    return convolve2d(x,kernel_v, mode='same', boundary='wrap')
def grad2d(x):
    """The full gradient operator: compute both x and y differences and return them all.  The x and y 
    differences are stacked so that rval[0] is a 2D array of x differences, and rval[1] is the y differences."""
    return np.stack([gradh(x),gradv(x)])

def gradht(x):
    """Adjoint of gradh"""
    kernel_ht = [[0,-1,1]] 
    return convolve2d(x,kernel_ht, mode='same', boundary='wrap')
def gradvt(x):
    """Adjoint of gradv"""
    kernel_vt = [[0],[-1],[1]]
    return convolve2d(x,kernel_vt, mode='same', boundary='wrap')
def divergence2d(x):
    "The method is the adjoint of grad2d."
    return gradht(x[0])+gradvt(x[1])

In [ ]:
# Testing

def h(z, eps=.01):
    """The hyperbolic approximation to L1"""
    return sum(sqrt(z*z+eps*eps).ravel())
def tv_denoise_objective(x,mu,b):
    return h(grad2d(x)) + 0.5*mu*norm(x-b)**2
def h_grad(z, eps=.01):
    """The gradient of h"""
    return z/sqrt(z*z+eps*eps)
def tv_denoise_grad(x,mu,b):
    """The gradient of the TV objective"""
    return divergence2d(h_grad(grad2d(x))) + mu*(x-b)

### Using the BB routine above to minimize the TV objective, and denoise the test image.

In [ ]:
# Generate a noisy image
image = zeros((50,50)).astype(np.uint8)
image[15:35,15:35]=1
#image = image+1*randn(50,50)
gauss_noise=np.zeros((image.shape[0],image.shape[1]),dtype=np.uint8)
cv2.randn(gauss_noise,128,20)
gauss_noise=(gauss_noise*0.03).astype(np.uint8)
image=cv2.add(image,gauss_noise)


plt.title('Noisy image')
plt.imshow(image,cmap='gray')
plt.show()

In [ ]:
# Generate a noisy image
image = zeros((200,200)).astype(np.uint8)
image[60:140,60:140]=1
image = image+1*randn(200,200)
#gauss_noise=np.zeros((image.shape[0],image.shape[1]),dtype=np.uint8)
#cv2.randn(gauss_noise,128,20)
#gauss_noise=(gauss_noise*0.03).astype(np.uint8)
#image=cv2.add(image,gauss_noise)


plt.title('Noisy image')
plt.imshow(image,cmap='gray')
plt.show()

In [ ]:
x0 = np.zeros(image.shape)
f = lambda x: tv_denoise_objective(x, mu=0.1, b=image)
grad = lambda x: tv_denoise_grad(x, mu=0.1, b=image)
#x= FR(f, grad, x0,0.001)
x, res = grad_descent_bb(f, grad, x0)
plt.title("Denoised image")
plt.imshow(x,cmap='gray')
plt.show()
print(f(x))

In [ ]:
f(x)# 0.2

In [ ]:
x0 = np.zeros(image.shape)
f = lambda x: tv_denoise_objective(x, mu=0.3, b=image)
grad = lambda x: tv_denoise_grad(x, mu=0.3, b=image)
x,_,k= FR(f, grad, x0,0.001)
print(k)
#x, res = grad_descent_bb(f, grad, x0)
plt.title("Denoised image")
plt.imshow(x,cmap='gray')
plt.show()

In [ ]:
def impulse_noise(image, prob):
    noisy_image = np.copy(image)
    salt_and_pepper = np.random.rand(*image.shape)
    noisy_image[salt_and_pepper < prob] = 0
    noisy_image[salt_and_pepper > 1 - prob] = 255
    return noisy_image

In [ ]:
# Generate a noisy image
Lena = cv2.imread('Lena.png')
Lena=cv2.cvtColor(Lena, cv2.COLOR_BGR2GRAY)
m,n=Lena.shape

Lena_noise = Lena+40*randn(m,n)

#Lena_noise=impulse_noise(Lena,0.03)

plt.title('Noisy image')
plt.imshow(Lena_noise,cmap='gray')
plt.show()

In [ ]:
x0 = np.zeros(Lena.shape)
f = lambda x: tv_denoise_objective(x, mu=0.01, b=Lena_noise)
grad = lambda x: tv_denoise_grad(x, mu=0.01, b=Lena_noise)
x,res,_= FR(f, grad, x0,0.001)
#x, res = grad_descent_bb(f, grad, x0,0.000001)
plt.title("Denoised image")
plt.imshow(x,cmap='gray')
plt.show()

In [ ]:
x0 = np.zeros(Lena.shape)
f = lambda x: tv_denoise_objective(x, mu=0.02, b=Lena_noise)
grad = lambda x: tv_denoise_grad(x, mu=0.02, b=Lena_noise)
x,res,_= FR(f, grad, x0,0.001)
#x, res = grad_descent_bb(f, grad, x0,0.000001)
plt.title("Denoised image")
plt.imshow(x,cmap='gray')
plt.show()

In [ ]:
x0 = np.zeros(Lena.shape)
f = lambda x: tv_denoise_objective(x, mu=0.03, b=Lena_noise)
grad = lambda x: tv_denoise_grad(x, mu=0.03, b=Lena_noise)
x,res,_= FR(f, grad, x0,0.001)
#x, res = grad_descent_bb(f, grad, x0,0.000001)
plt.title("Denoised image")
plt.imshow(x,cmap='gray')
plt.show()

In [ ]:
x0 = np.zeros(Lena.shape)
f = lambda x: tv_denoise_objective(x, mu=0.05, b=Lena_noise)
grad = lambda x: tv_denoise_grad(x, mu=0.05, b=Lena_noise)
#x,res,_= FR(f, grad, x0,0.001)
x, res = grad_descent_bb(f, grad, x0,0.001)
plt.title("Denoised image")
plt.imshow(x,cmap='gray')
plt.show()

In [ ]:
x=x.astype(np.uint8)
Lena_noise=Lena_noise.astype(np.uint8)


In [ ]:
psnr = cv2.PSNR(x, Lena_noise)
psnr

In [ ]:
def impulse_noise(image, prob):
    noisy_image = np.copy(image)
    salt_and_pepper = np.random.rand(*image.shape)
    noisy_image[salt_and_pepper < prob] = 0
    noisy_image[salt_and_pepper > 1 - prob] = 255
    return noisy_image

In [ ]:
def FR_1(f,grad,x0,e=0.001):
    
    xcur = np.array(x0)
    n = len(x0)
    k = 0 # step1
    dk=grad(x0)
    prevgrad = 1
    pk = -1*dk
    res=[]
    
    prevx=0
    #while (np.linalg.norm(prevx-x)>e): # step3
    while (k<1000): # step3
        if (k%n==0): # step4
            pk = -1*dk
        else:
            bk = (np.linalg.norm(dk)**2)/(np.linalg.norm(prevgrad)**2) # step5
            prevpk = pk
            pk = -1*dk + bk*prevpk # step6
        a = (optimize.minimize_scalar(lambda x: f(xcur+pk*x), bounds=(0,10)).x)
        prevx=xcur
        xcur = xcur + a*pk #step8
        
        k=k+1 #step8
        prevgrad=dk
        dk=grad(xcur)
        res.append(xcur)
    return xcur,res,k #step10

In [ ]:
# Generate a noisy image
img = zeros((50,50)).astype(np.uint8)
img[15:35,15:35]=1
#image = image+1*randn(50,50)
gauss_noise=np.zeros((img.shape[0],img.shape[1]),dtype=np.uint8)
cv2.randn(gauss_noise,128,20)
gauss_noise=(gauss_noise*0.02).astype(np.uint8)
img=cv2.add(img,gauss_noise)


plt.title('Noisy image')
plt.imshow(img,cmap='gray')
plt.show()

In [ ]:
x0 = np.zeros(img.shape)
f = lambda x: tv_denoise_objective(x, mu=0.3, b=img)
grad = lambda x: tv_denoise_grad(x, mu=0.3, b=img)
x,results,k= FR_1(f, grad, x0,0.001)

for res in results:
    plt.title("Denoised image")
    plt.imshow(res,cmap='gray')
    plt.show()
    
    
print(k)